In [3]:
!pip install sounddevice

In [76]:
import  tensorflow_io as tfio
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sounddevice as sd
import wave

In [113]:
def record_audio(file_name, duration, sample_rate=44100):
    """
    Records audio from the default microphone and saves it as a .wav file.

    Parameters:
        file_name (str): Name of the file to save (e.g., "output.wav").
        duration (int): Duration of the recording in seconds.
        sample_rate (int): Sample rate for the recording (default is 44100 Hz).
    """
    print(f"Recording for {duration} seconds...")
    
    # Record audio
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()  # Wait for the recording to complete
    print("Recording complete.")
    
    # Save the recorded audio to a .wav file
    with wave.open(file_name, 'wb') as wf:
        wf.setnchannels(1)  # Mono channel
        wf.setsampwidth(2)  # 2 bytes per sample (16-bit audio)
        wf.setframerate(sample_rate)
        wf.writeframes(audio_data.tobytes())
    
    print(f"Audio saved as {file_name}.")

In [126]:
output_file = "output.wav"
record_duration = 1  # Duration in seconds
record_audio(output_file, record_duration)

Recording for 1 seconds...
Recording complete.
Audio saved as output.wav.


In [127]:
def mono_16k(path):
    file_contents = tf.io.read_file(path)
    wav ,sample_rate = tf.audio.decode_wav(file_contents, desired_channels = 1)
    wav = tf.squeeze(wav, axis = -1)
    sample_rate = tf.cast(sample_rate, dtype= tf.int64)
    #wav = tfio.audio.resample(wav, rate_in = sample_rate, rate_out = 16000)
    return wav

In [128]:
path='output.wav'
k=mono_16k(path)
print(k)
tf.shape(k)

tf.Tensor([0.00015259 0.00027466 0.00027466 ... 0.00067139 0.0007019  0.00067139], shape=(44100,), dtype=float32)


<tf.Tensor: shape=(1,), dtype=int32, numpy=array([44100], dtype=int32)>

In [129]:
def make_spectrogram(wav):
  input_len = 16000
  wav = wav[:input_len]
  zero_padding = tf.zeros([16000] - tf.shape(wav),dtype=tf.float32)
  wav = tf.cast(wav, dtype=tf.float32)
  equal_length = tf.concat([wav, zero_padding], 0)
  spectrogram = tf.signal.stft(equal_length, frame_length=320, frame_step=32)
  spectrogram = tf.abs(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis =2)


  return spectrogram

spectogram=make_spectrogram(k)

In [130]:
spectogram=tf.reshape(spectogram, (491, 1, 257))
tf.shape(spectogram)


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([491,   1, 257], dtype=int32)>

In [131]:
model=tf.keras.models.load_model('Model_23M_9089.keras')

In [132]:
config = model.get_config() # Returns pretty much every information about your model
print(config["layers"][0]["config"]["batch_shape"])
print(tf.shape(spectogram))

(None, 491, 1, 257)
tf.Tensor([491   1 257], shape=(3,), dtype=int32)


In [133]:
spectogram = spectogram[tf.newaxis,...]

In [134]:
prediction=model.predict(spectogram)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [135]:
o=max(prediction[0])
print(o)
prediction[0]
ind_np = ((np.where(prediction[0] ==o ))[0])
index = (ind_np[0])

0.80324745


In [136]:
labels=['eight', 'nine', 'three', 'one', 'zero', 'seven', 'two', 'six', 'five', 'four']
print(labels[index])

five
